### 0. CONFIGURACIÓN INICIAL 
Importamos librerias, creamos rutas de las carpetas que se utilizaran, adicionalmente se verifica que exitan los archivos a trabajar

In [2]:
# Importamos las librerías necesarias
import pandas as pd
import numpy as np
import os

# Configuraciones generales de pandas
pd.set_option('display.max_columns', None)
pd.set_option('display.precision', 2)

# Definimos las rutas base de los datos  brutos y ya limpios
ruta_datos = r"C:\Estudios\Talento_Tech\Proyecto_Talento_Tech\proyecto_movilidad_electrica\datos\brutos"
ruta_salida = r"C:\Estudios\Talento_Tech\Proyecto_Talento_Tech\proyecto_movilidad_electrica\datos\limpios"


# Verificamos que las rutas existan
os.makedirs(ruta_salida, exist_ok=True)

print("Rutas configuradas correctamente ✅")

def mostrar_resumen_df(df, n=3):
    """Resumen rápido de un DataFrame."""
    print(f"Filas: {len(df):,} | Columnas: {len(df.columns)}")
    display(df.head(n))
    print("\nTipos de datos:")
    print(df.dtypes)

def normalizar_texto(serie):
    """Estandariza textos (departamento/municipio)."""
    return (serie.astype(str)
                 .str.strip()
                 .str.upper())

def limpiar_anio(serie):
    """Convierte '2,022' → 2022 (entero)."""
    return (serie.astype(str)
                 .str.replace(",", "", regex=False)
                 .str.extract(r"(\d{4})")[0]
                 .astype("Int64"))

def a_numero_seguro(serie, quitar_miles=True, coma_decimal=False):
    """
    Convierte strings a numérico manejando separadores (miles/decimal).
    - quitar_miles=True elimina ',' y '.'
    - coma_decimal=True trata ',' como decimal (convierte a '.')
    """
    s = serie.astype(str).strip()
    if coma_decimal:
        s = s.replace(".", "", regex=False).replace(",", ".", regex=False)
    elif quitar_miles:
        s = s.replace(",", "", regex=False).replace(".", "", regex=False)
    return pd.to_numeric(s, errors="coerce")

# ========================================

# --- LIMPIEZA DE DEPARTAMENTOS (DIVIPOLA) USANDO POSICIÓN DE COLUMNAS ---

# 1️⃣ Cargar el archivo
archivo_divipola = os.path.join(ruta_datos, "DIVIPOLA-_C_digos_municipios.csv")
df_divipola = pd.read_csv(archivo_divipola, encoding="latin1")

print("Archivo DIVIPOLA cargado correctamente ✅")
mostrar_resumen_df(df_divipola)

# 2️⃣ Tomamos las dos primeras columnas (por posición)
df_departamentos = df_divipola.iloc[:, [0, 1]].copy()
df_departamentos.columns = ['COD_DEPTO', 'DEPARTAMENTO']

# 3️⃣ Corregimos problemas de tildes y eñes
def arreglar_tildes(texto):
    """Corrige textos mal codificados (acentos y eñes)."""
    if isinstance(texto, str):
        texto = (texto
                 .encode('latin1', errors='ignore')
                 .decode('utf-8', errors='ignore')
                 .strip()
                 .upper())
        # Reforzamos reemplazos comunes
        reemplazos = {
            "ATLNTICO": "ATLÁNTICO",
            "BOLVAR": "BOLÍVAR",
            "BOYAC": "BOYACÁ",
            "QUINDO": "QUINDÍO",
            "NARIO": "NARIÑO",
            "SAN ANDRS": "SAN ANDRÉS",
            "VALLE DEL CAUCA": "VALLE DEL CAUCA",
        }
        for k, v in reemplazos.items():
            texto = texto.replace(k, v)
        return texto
    return texto

df_departamentos['DEPARTAMENTO'] = df_departamentos['DEPARTAMENTO'].apply(arreglar_tildes)

# 4️⃣ Aseguramos formato del código (2 dígitos)
df_departamentos['COD_DEPTO'] = df_departamentos['COD_DEPTO'].astype(str).str.zfill(2)

# 5️⃣ Eliminamos duplicados y ordenamos
df_departamentos = (
    df_departamentos.drop_duplicates()
    .sort_values('COD_DEPTO')
    .reset_index(drop=True)
)

# 6️⃣ Mostramos resultado final
print("\n✅ Departamentos únicos y estandarizados:")
mostrar_resumen_df(df_departamentos)

# 7️⃣ Guardamos el archivo limpio
ruta_salida_departamentos = os.path.join(ruta_salida, "departamentos_limpios.csv")
df_departamentos.to_csv(ruta_salida_departamentos, index=False, encoding="utf-8-sig")

print(f"\nArchivo limpio guardado en: {ruta_salida_departamentos}")



Rutas configuradas correctamente ✅
Archivo DIVIPOLA cargado correctamente ✅
Filas: 1,122 | Columnas: 7


,CÃ³digo Departamento,Nombre Departamento,CÃ³digo Municipio,Nombre Municipio,Tipo: Municipio / Isla / Ãrea no municipalizada,longitud,Latitud
0,5,ANTIOQUIA,5001,MEDELLÃN,Municipio,"-75,581775","6,246631"
1,5,ANTIOQUIA,5002,ABEJORRAL,Municipio,"-75,428739","5,789315"
2,5,ANTIOQUIA,5004,ABRIAQUÃ,Municipio,"-76,064304","6,632282"



Tipos de datos:
CÃ³digo Departamento                                 int64
Nombre Departamento                                 object
CÃ³digo Municipio                                    int64
Nombre Municipio                                    object
Tipo: Municipio / Isla / Ãrea no municipalizada    object
longitud                                            object
Latitud                                             object
dtype: object

✅ Departamentos únicos y estandarizados:
Filas: 33 | Columnas: 2


,COD_DEPTO,DEPARTAMENTO
0,05,ANTIOQUIA
1,08,ATLÁNTICO
2,11,"BOGOTÁ, D.C."



Tipos de datos:
COD_DEPTO       object
DEPARTAMENTO    object
dtype: object

Archivo limpio guardado en: C:\Estudios\Talento_Tech\Proyecto_Talento_Tech\proyecto_movilidad_electrica\datos\limpios\departamentos_limpios.csv



### 1. CARGA Y LIMPIEZA DE VEHICULOS ELECTRICOS E HIBRIDOS


In [3]:
# ================================
# INGESTA Y LIMPIEZA: VEHÍCULOS (EV/HEV)
# ================================

import os
import unicodedata
import numpy as np
import pandas as pd

# ---------- 0) Helpers específicos de este script ----------
def clave_union_dep(serie: pd.Series) -> pd.Series:
    """
    Crea una clave robusta para unir departamentos:
    - usa normalizar_texto (mayúsculas, strip)
    - quita diacríticos (tildes/ñ -> n)
    - elimina caracteres no alfabéticos (deja letras y espacios)
    - colapsa espacios múltiples
    """
    s = normalizar_texto(serie).fillna("")
    s = s.apply(lambda x: ''.join(ch for ch in unicodedata.normalize('NFKD', x)
                                  if not unicodedata.combining(ch)))
    s = s.str.replace(r"[^A-Z\s]", "", regex=True)
    s = s.str.replace(r"\s+", " ", regex=True).str.strip()
    return s

# ---------- 1) Carga segura del archivo en bruto ----------
ruta_ev = os.path.join(ruta_datos, "Numero_de_Vehículos_Eléctricos_-_Hibridos_20251009.csv")

vehiculos_raw = pd.read_csv(
    ruta_ev,
    dtype={
        "AÑO_REGISTRO": "string",
        "FECHA_REGISTRO": "string",
        "DEPARTAMENTO": "string",
        "MUNICIPIO": "string",
        "COMBUSTIBLE": "string",
        "CLASIFICACION": "string",
        "CLASE": "string",
        "SERVICIO": "string",
        "MARCA": "string"
    },
    low_memory=False
)

print("✅ Archivo de vehículos cargado (bruto).")
mostrar_resumen_df(vehiculos_raw, n=5)

# ---------- 2) Renombrado a nombres genéricos y selección ----------
mapeo_columnas = {
    "COMBUSTIBLE": "combustible",
    "FECHA_REGISTRO": "fecha_registro",
    "AÑO_REGISTRO": "anio_registro",
    "CLASIFICACION": "clasificacion",
    "CLASE": "clase",
    "SERVICIO": "servicio",
    "MARCA": "marca",
    "MUNICIPIO": "municipio",
    "DEPARTAMENTO": "departamento"
}

vehiculos = vehiculos_raw.copy()
# Si alguna columna no existe, se crea vacía para no romper el flujo
for col_ori in mapeo_columnas:
    if col_ori not in vehiculos.columns:
        vehiculos[col_ori] = pd.NA

vehiculos = vehiculos[list(mapeo_columnas.keys())].rename(columns=mapeo_columnas)

# ---------- 3) Limpieza de año de registro ----------
vehiculos["anio_registro"] = limpiar_anio(vehiculos["anio_registro"])

# Si hay años faltantes, intentamos extraer desde 'fecha_registro'
if vehiculos["anio_registro"].isna().any():
    extra_anio = vehiculos["fecha_registro"].astype("string").str.extract(r"(\d{4})")[0].astype("Int64")
    vehiculos["anio_registro"] = vehiculos["anio_registro"].fillna(extra_anio)

# ---------- 4) Estandarización de texto en variables categóricas ----------
cols_texto = ["departamento", "municipio", "combustible", "clasificacion", "clase", "servicio", "marca"]
for c in cols_texto:
    vehiculos[c] = normalizar_texto(vehiculos[c])

# ================================
# 4A) Armonización de DEPARTAMENTO con referencia DIVIPOLA
# ================================
ref_path = os.path.join(ruta_salida, "departamentos_limpios.csv")
ref_deps = pd.read_csv(ref_path, encoding="utf-8-sig")

# Claves de unión (mismo método en ambos lados)
ref_deps["dep_join"]  = clave_union_dep(ref_deps["DEPARTAMENTO"])
vehiculos["dep_join"] = clave_union_dep(vehiculos["departamento"])

# === Alias de unión (APLICAR ANTES DEL MERGE) ===
alias_map = {
    # San Andrés (formas comunes en fuentes operativas)
    "SAN ANDRES": "ARCHIPIELAGO DE SAN ANDRES PROVIDENCIA Y SANTA CATALINA",
    "SAN ANDRES ISLAS": "ARCHIPIELAGO DE SAN ANDRES PROVIDENCIA Y SANTA CATALINA",
    "ARCHIPIELAGO DE SAN ANDRES": "ARCHIPIELAGO DE SAN ANDRES PROVIDENCIA Y SANTA CATALINA",
    # Boyacá (variantes con prefijos)
    "DEPTO DE BOYACA": "BOYACA",
    "DEPARTAMENTO DE BOYACA": "BOYACA",
    "DPTO BOYACA": "BOYACA"
}
vehiculos["dep_join"] = vehiculos["dep_join"].replace(alias_map)

# (Opcional robusto) Duplicar filas alias en la referencia para cubrir más casos
alias_rows = pd.DataFrame({
    "dep_join": [
        "SAN ANDRES", "SAN ANDRES ISLAS", "ARCHIPIELAGO DE SAN ANDRES", "BOYACA"
    ],
    "COD_DEPTO": ["88","88","88","15"],
    "DEPARTAMENTO": [
        "ARCHIPIÉLAGO DE SAN ANDRÉS, PROVIDENCIA Y SANTA CATALINA",
        "ARCHIPIÉLAGO DE SAN ANDRÉS, PROVIDENCIA Y SANTA CATALINA",
        "ARCHIPIÉLAGO DE SAN ANDRÉS, PROVIDENCIA Y SANTA CATALINA",
        "BOYACÁ"
    ]
})
ref_deps = pd.concat([ref_deps, alias_rows], ignore_index=True)
ref_deps = ref_deps.drop_duplicates(subset=["dep_join"], keep="first")

# === MERGE (después de alias) ===
vehiculos = vehiculos.merge(
    ref_deps[["dep_join", "COD_DEPTO", "DEPARTAMENTO"]],
    on="dep_join",
    how="left",
    suffixes=("", "_ref")     # por si alguna vez coincidiera el nombre
)

# ---------- Diagnóstico de no mapeados (ANTES de formatear COD_DEPTO) ----------
no_mapeados = vehiculos[vehiculos["COD_DEPTO"].isna()]["departamento"].dropna().unique()
if len(no_mapeados) > 0:
    print("\n⚠️ Departamentos no mapeados (revisar y, si aplica, actualizar referencia):")
    for d in no_mapeados:
        print("-", d)
else:
    print("\n✅ Todos los departamentos fueron armonizados con la referencia DIVIPOLA.")

# ---------- Formateo robusto de COD_DEPTO ----------
# 1) convertir tolerante a numérico (cadenas/ruido -> NaN), preservando NaN reales
vehiculos["COD_DEPTO"] = pd.to_numeric(vehiculos["COD_DEPTO"], errors="coerce")
# 2) ahora sí a Int64 (permite NaN) y a 2 dígitos
vehiculos["COD_DEPTO"] = (
    vehiculos["COD_DEPTO"]
      .astype("Int64")
      .astype("string")
      .str.zfill(2)
)

# ---------- Tomar el nombre estándar sin romper si cambia el sufijo ----------
# En la mayoría de casos la columna se llama exactamente "DEPARTAMENTO" (la traída del ref).
# Si por alguna razón vino con sufijo, tomamos el que exista.
col_std = None
for cand in ["DEPARTAMENTO", "DEPARTAMENTO_ref", "DEPARTAMENTO_std"]:
    if cand in vehiculos.columns:
        col_std = cand
        break

if col_std is not None:
    vehiculos["departamento"] = vehiculos[col_std]
# eliminar auxiliares sin romper si no existen
vehiculos = vehiculos.drop(columns=[c for c in ["DEPARTAMENTO", "DEPARTAMENTO_ref", "DEPARTAMENTO_std", "dep_join"] if c in vehiculos.columns])





# ---------- 5) Clasificación de TIPO_VEHICULO (EV / HEV / OTRO) ----------
comb = vehiculos["combustible"].fillna("")

es_hev = (
    comb.str.contains(r"\bHIB", regex=True) |
    (comb.str.contains(r"GAS", regex=True) & comb.str.contains(r"ELEC", regex=True)) |
    (comb.str.contains(r"DIE", regex=True) & comb.str.contains(r"ELEC", regex=True)) |
    comb.str.contains(r"\bHEV\b", regex=True) |
    comb.str.contains(r"\bPHEV\b", regex=True)
)

es_ev = comb.str.contains(r"ELEC", regex=True) & ~es_hev

vehiculos["tipo_vehiculo"] = np.where(es_hev, "HEV", np.where(es_ev, "EV", "OTRO"))

# ---------- 6) Orden de columnas (sin romper si falta alguna) ----------
orden_deseado = [
    "tipo_vehiculo", "combustible", "anio_registro", "fecha_registro",
    "clasificacion", "clase", "servicio", "marca", "municipio", "departamento", "COD_DEPTO"
]
orden_final = [c for c in orden_deseado if c in vehiculos.columns]
vehiculos = vehiculos[orden_final]

faltantes = [c for c in orden_deseado if c not in vehiculos.columns]
if faltantes:
    print("ℹ️ Columnas no presentes al reordenar (no se incluyen):", faltantes)

# ---------- 7) Chequeos rápidos ----------
print("\n=== Chequeos rápidos ===")
print("Rango de años:", vehiculos["anio_registro"].min(), "->", vehiculos["anio_registro"].max())
print("Departamentos únicos:", vehiculos["departamento"].nunique())
print("Municipios únicos:", vehiculos["municipio"].nunique())
print("Distribución por tipo_vehiculo (%):")
print((vehiculos["tipo_vehiculo"].value_counts(dropna=False, normalize=True) * 100).round(2))

# ---------- 8) Vista previa del archivo limpio ----------
print("\n✅ Vista previa del archivo limpio:")
mostrar_resumen_df(vehiculos.sample(min(10, len(vehiculos))), n=10)

# ---------- 9) Guardado del archivo limpio ----------
nombre_salida = "vehiculos_ev_hev_limpio.csv"
ruta_salida_archivo = os.path.join(ruta_salida, nombre_salida)
vehiculos.to_csv(ruta_salida_archivo, index=False, encoding="utf-8-sig")
print(f"\n💾 Archivo limpio guardado en: {ruta_salida_archivo}")



✅ Archivo de vehículos cargado (bruto).
Filas: 56,545 | Columnas: 22


,COMBUSTIBLE,ESTADO,MODELO,FECHA_REGISTRO,AÑO_REGISTRO,CLASIFICACION,CLASE,SERVICIO,MARCA,LINEA,CARROCERIA,CILINDRAJE,MODALIDAD,ORGANISMO_TRANSITO,MUNICIPIO,DEPARTAMENTO,CAPACIDAD_CARGA,CAPACIDAD_PASAJEROS,PESO,POTENCIA,EJES,CANTIDAD
0,ELECTRICO,ACTIVO,"2,022",2022 Jun 30 12:00:00 AM,"2,022",AUTOMOVIL,BUS,Público,BYD,BC11S01,CERRADA,NaN,PASAJEROS,SDM - BOGOTA D.C.,BOGOTA,Bogota D.C.,NaN,49.0,"20,000",402,2.0,1
1,ELECTRICO,ACTIVO,"2,023",2022 Oct 21 12:00:00 AM,"2,022",AUTOMOVIL,CAMIONETA,Particular,BYD,YUAN PRO EV,WAGON,0.0,NaN,INSTITUTO DE MOVILIDAD DE PEREIRA,PEREIRA,Risaralda,NaN,NaN,"1,980",134,2.0,1
2,ELECTRICO,ACTIVO,"2,014",2015 Sep 28 12:00:00 AM,"2,015",MOTO,MOTOCICLETA,Particular,E-MOTORI,VITA,SIN CARROCERIA,0.0,NaN,STRIA TTOyTTE MCPAL FLORENCIA,FLORENCIA,Caqueta,NaN,NaN,NaN,NaN,NaN,1
3,ELECTRICO,ACTIVO,"2,021",2022 Aug 10 12:00:00 AM,"2,022",AUTOMOVIL,CAMIONETA,Público,DONGFENG,DFA5030XXYABEV7,PANEL,NaN,CARGA,STRIA TTOyTTE MCPAL FUNZA,FUNZA,Cundinamarca,845,NaN,"2,550",80,NaN,1
4,ELECTRICO,ACTIVO,"2,022",2021 Oct 25 12:00:00 AM,"2,021",AUTOMOVIL,CAMIONETA,Particular,BYD,SONG PRO EV,WAGON,0.0,NaN,STRIA TTEyMOV CUND/EL ROSAL,EL ROSAL,Cundinamarca,NaN,NaN,"2,120",161,2.0,1



Tipos de datos:
COMBUSTIBLE            string[python]
ESTADO                         object
MODELO                         object
FECHA_REGISTRO         string[python]
AÑO_REGISTRO           string[python]
CLASIFICACION          string[python]
CLASE                  string[python]
SERVICIO               string[python]
MARCA                  string[python]
LINEA                          object
CARROCERIA                     object
CILINDRAJE                    float64
MODALIDAD                      object
ORGANISMO_TRANSITO             object
MUNICIPIO              string[python]
DEPARTAMENTO           string[python]
CAPACIDAD_CARGA                object
CAPACIDAD_PASAJEROS           float64
PESO                           object
POTENCIA                       object
EJES                          float64
CANTIDAD                        int64
dtype: object

⚠️ Departamentos no mapeados (revisar y, si aplica, actualizar referencia):
- ARCHIPIELAGO DE SAN ANDRES, PROVIDENCIA

=== Chequeos 

,tipo_vehiculo,combustible,anio_registro,fecha_registro,clasificacion,clase,servicio,marca,municipio,departamento,COD_DEPTO
9369,EV,ELECTRICO,2022,2022 May 31 12:00:00 AM,AUTOMOVIL,BUS,PÚBLICO,BYD,BOGOTA,"BOGOTÁ, D.C.",11
11304,HEV,GASO ELEC,2022,2022 Jun 03 12:00:00 AM,AUTOMOVIL,CAMIONETA,PARTICULAR,TOYOTA,ENVIGADO,ANTIOQUIA,05
41973,HEV,GASO ELEC,2021,2021 Aug 27 12:00:00 AM,AUTOMOVIL,CAMIONETA,PARTICULAR,TOYOTA,BUCARAMANGA,SANTANDER,68
14245,HEV,GASO ELEC,2021,2021 Nov 09 12:00:00 AM,AUTOMOVIL,CAMIONETA,PARTICULAR,FORD,BOGOTA,"BOGOTÁ, D.C.",11
42595,HEV,GASO ELEC,2022,2022 Mar 10 12:00:00 AM,AUTOMOVIL,AUTOMOVIL,PARTICULAR,TOYOTA,VILLA DEL ROSARIO,NORTE DE SANTANDER,54
37405,HEV,GASO ELEC,2022,2022 Apr 05 12:00:00 AM,AUTOMOVIL,CAMIONETA,PARTICULAR,MAZDA,IBAGUE,TOLIMA,73
23883,HEV,GASO ELEC,2021,2021 Jul 02 12:00:00 AM,AUTOMOVIL,CAMPERO,PARTICULAR,SUBARU,BOGOTA,"BOGOTÁ, D.C.",11
17000,HEV,GASO ELEC,2021,2021 Jun 03 12:00:00 AM,AUTOMOVIL,CAMIONETA,PARTICULAR,KIA,BOGOTA,"BOGOTÁ, D.C.",11
25556,HEV,GASO ELEC,2022,2022 May 24 12:00:00 AM,AUTOMOVIL,CAMIONETA,PARTICULAR,MAZDA,CAJICA,CUNDINAMARCA,25
48422,HEV,GASO ELEC,2022,2022 Mar 22 12:00:00 AM,AUTOMOVIL,CAMPERO,PARTICULAR,TOYOTA,SOACHA,CUNDINAMARCA,25



Tipos de datos:
tipo_vehiculo             object
combustible               object
anio_registro              Int64
fecha_registro    string[python]
clasificacion             object
clase                     object
servicio                  object
marca                     object
municipio                 object
departamento              object
COD_DEPTO         string[python]
dtype: object

💾 Archivo limpio guardado en: C:\Estudios\Talento_Tech\Proyecto_Talento_Tech\proyecto_movilidad_electrica\datos\limpios\vehiculos_ev_hev_limpio.csv


### 2. CARGA, LIMPIEZA y AJUSTE: PIB DEPARTAMENTAL
Lee línea por línea, toma solo el primer segmento antes del ; (el resto son “rellenos”).
Limpia comillas escapadas y parsea con csv.reader (respeta comillas internas).
Reconstruye el año cuando viene como ["2","005"].
Filtra filas basura y normaliza anio y valor.
Filtra “constantes” para quedarte solo con precios constantes 2015.
Agrega por departamento y anio y deja la columna final como pib_const_2015_mil_mm.

In [4]:
# =========================================
# PIB DEPARTAMENTAL (CONST. 2015) – Limpieza, unificación y export seguro
# Salida: anio, codigo_depto, departamento, pib_const_2015_miles_mm
# =========================================
import os, re, csv, unicodedata
import pandas as pd
from datetime import datetime

# ------- Rutas (ajusta a tu proyecto) ----
ruta_pib = os.path.join(ruta_datos, "PIB_Departamental_con_proyección_20251014.csv")
out_base = os.path.join(ruta_salida, "pib_departamental_const2015_limpio")

# ------- Catálogo canónico DANE ----------
CANON_DEPTOS = [
    ("05","ANTIOQUIA"), ("08","ATLÁNTICO"), ("11","BOGOTÁ D.C."), ("13","BOLÍVAR"),
    ("15","BOYACÁ"), ("17","CALDAS"), ("18","CAQUETÁ"), ("19","CAUCA"),
    ("20","CESAR"), ("23","CÓRDOBA"), ("25","CUNDINAMARCA"), ("27","CHOCÓ"),
    ("41","HUILA"), ("44","LA GUAJIRA"), ("47","MAGDALENA"), ("50","META"),
    ("52","NARIÑO"), ("54","NORTE DE SANTANDER"), ("63","QUINDÍO"), ("66","RISARALDA"),
    ("68","SANTANDER"), ("70","SUCRE"), ("73","TOLIMA"), ("76","VALLE DEL CAUCA"),
    ("81","ARAUCA"), ("85","CASANARE"), ("86","PUTUMAYO"),
    ("88","SAN ANDRÉS, PROVIDENCIA Y SANTA CATALINA"),
    ("91","AMAZONAS"), ("94","GUAINÍA"), ("95","GUAVIARE"), ("97","VAUPÉS"), ("99","VICHADA")
]
CODE2NAME = {c:n for c,n in CANON_DEPTOS}
VARIANTES_NOMBRE = {
    "ANTIOQUIA": ("05","ANTIOQUIA"),
    "ATLANTICO": ("08","ATLÁNTICO"),
    "BOGOTA D.C": ("11","BOGOTÁ D.C."), "BOGOTA DC": ("11","BOGOTÁ D.C."), "BOGOTA": ("11","BOGOTÁ D.C."),
    "BOLIVAR": ("13","BOLÍVAR"), "BOYACA": ("15","BOYACÁ"), "CALDAS": ("17","CALDAS"),
    "CAQUETA": ("18","CAQUETÁ"), "CAUCA": ("19","CAUCA"), "CESAR": ("20","CESAR"),
    "CORDOBA": ("23","CÓRDOBA"), "CUNDINAMARCA": ("25","CUNDINAMARCA"), "CHOCO": ("27","CHOCÓ"),
    "HUILA": ("41","HUILA"), "LA GUAJIRA": ("44","LA GUAJIRA"), "MAGDALENA": ("47","MAGDALENA"),
    "META": ("50","META"), "NARINO": ("52","NARIÑO"), "NORTE DE SANTANDER": ("54","NORTE DE SANTANDER"),
    "N. DE SANTANDER": ("54","NORTE DE SANTANDER"),
    "QUINDIO": ("63","QUINDÍO"), "RISARALDA": ("66","RISARALDA"), "SANTANDER": ("68","SANTANDER"),
    "SUCRE": ("70","SUCRE"), "TOLIMA": ("73","TOLIMA"),
    "VALLE": ("76","VALLE DEL CAUCA"), "VALLE DEL CAUCA": ("76","VALLE DEL CAUCA"),
    "ARAUCA": ("81","ARAUCA"), "CASANARE": ("85","CASANARE"), "PUTUMAYO": ("86","PUTUMAYO"),
    "SAN ANDRES": ("88","SAN ANDRÉS, PROVIDENCIA Y SANTA CATALINA"),
    "SAN ANDRES PROVIDENCIA Y SANTA CATALINA": ("88","SAN ANDRÉS, PROVIDENCIA Y SANTA CATALINA"),
    "ARCHIPIELAGO DE SAN ANDRES PROVIDENCIA Y SANTA CATALINA": ("88","SAN ANDRÉS, PROVIDENCIA Y SANTA CATALINA"),
    "AMAZONAS": ("91","AMAZONAS"), "GUAINIA": ("94","GUAINÍA"), "GUAVIARE": ("95","GUAVIARE"),
    "VAUPES": ("97","VAUPÉS"), "VICHADA": ("99","VICHADA"),
}

# ------- Utilidades de texto -------------
def _normalize_text_value(s):
    if pd.isna(s): return pd.NA
    s = str(s).strip().upper()
    s = unicodedata.normalize("NFKD", s).encode("ascii", "ignore").decode("ascii")
    s = re.sub(r"\s+", " ", s)
    s = s.replace("D.C.", "D.C").replace("D C", "D.C")
    return s

def normalize_text(x):
    return x.map(_normalize_text_value) if isinstance(x, pd.Series) else _normalize_text_value(x)

def canon_dep(cod, nombre):
    cod2 = None
    if cod:
        m = re.search(r"\d+", str(cod))
        if m:
            c = m.group(0).zfill(2)
            if c in CODE2NAME:
                return c, CODE2NAME[c]
            cod2 = c
    nom_norm = normalize_text(nombre) if nombre is not None else ""
    if nom_norm in VARIANTES_NOMBRE:
        return VARIANTES_NOMBRE[nom_norm]
    return (cod2 if cod2 else ""), (nombre if nombre is not None else "")

# ------- Lectura del CSV “raro” ----------
rows = []
with open(ruta_pib, "r", encoding="utf-8", errors="replace") as f:
    for line in f:
        inner = line.strip().split(";")[0]
        if inner.startswith('"') and inner.endswith('"'): inner = inner[1:-1]
        inner = inner.replace('""', '"')
        parsed = next(csv.reader([inner], quotechar='"', delimiter=',', skipinitialspace=True))
        rows.append(parsed)

def parse_rows_to_df(rows):
    data = []
    for r in rows:
        if not r: continue
        if any(h in r[0] for h in ["A\u00f1o","AÑO","Año","Tipo de precios","Departamento","Valor"]):
            continue
        if len(r)==8 and r[0].isdigit() and r[1].isdigit():
            r = [r[0]+r[1]] + r[2:8]
        elif len(r)>=7:
            r = r[:7]
        else:
            continue
        data.append(r)
    return pd.DataFrame(data, columns=["anio","actividad","sector","tipo_precio","codigo_depto","departamento","valor_miles_millones"])

pib = parse_rows_to_df(rows)

# ------- Limpieza numérica ---------------
def clean_num(s: str):
    if pd.isna(s): return pd.NA
    s = str(s).strip()
    if s == "": return pd.NA
    s = re.sub(r"[^0-9,.\-]", "", s)
    if "," in s and "." in s:
        last = max(s.rfind(","), s.rfind("."))
        int_part = re.sub(r"[.,]", "", s[:last])
        frac_part = re.sub(r"[.,]", "", s[last+1:])
        s = f"{int_part}.{frac_part}" if frac_part != "" else f"{int_part}.0"
        try: return float(s)
        except: return pd.NA
    if "," in s and "." not in s:
        if s.count(",")==1 and len(s.split(",")[-1]) in (1,2,3): s = s.replace(",", ".")
        else: s = s.replace(",", "")
        try: return float(s)
        except: return pd.NA
    if "." in s and "," not in s:
        if s.count(".")==1:
            try: return float(s)
            except: return pd.NA
        last = s.rfind(".")
        if len(s)-last-1 in (1,2,3):
            s = f"{s[:last].replace('.','')}.{s[last+1:]}"
        else:
            s = s.replace(".", "")
        try: return float(s)
        except: return pd.NA
    try: return float(s)
    except: return pd.NA

# ---- Tipos + unificación de deptos ------
pib["anio"] = pd.to_numeric(pib["anio"], errors="coerce").astype("Int64")
pib["codigo_depto"] = (pib["codigo_depto"].astype(str).str.extract(r"(\d+)", expand=False).fillna("").str.zfill(2))

canon = pib.apply(lambda r: canon_dep(r["codigo_depto"], r["departamento"]), axis=1, result_type="expand")
canon.columns = ["codigo_depto_canon","departamento_canon"]
pib["codigo_depto"] = canon["codigo_depto_canon"]
pib["departamento"] = canon["departamento_canon"]

pib["valor_miles_millones"] = pd.to_numeric(pib["valor_miles_millones"].map(clean_num), errors="coerce")

# ---- Filtro: constantes 2015 ------------
mask = pib["tipo_precio"].str.contains("constantes", case=False, na=False) & pib["tipo_precio"].str.contains("2015", case=False, na=False)
pib_c2015 = pib[mask].copy()

# ---- Agregación --------------------------
pib_limpio = (pib_c2015
    .groupby(["anio","codigo_depto","departamento"], as_index=False, dropna=False)["valor_miles_millones"]
    .sum(min_count=1)
    .rename(columns={"valor_miles_millones":"pib_const_2015_miles_mm"})
    .sort_values(["anio","departamento"], kind="stable")
)

# ---- Sanidad numérica FINAL (clave) -----
def coerce_final_num(x):
    """Si aún quedara texto, lo limpia a número sin separadores de miles."""
    if pd.isna(x): return pd.NA
    if isinstance(x, (int, float)): return float(x)
    s = str(x)
    s = s.replace("\u00A0","").replace(" ","")
    # quitar separadores de miles (cualquier . o , que NO sea el último separador decimal)
    # regla: el último separador con 1–3 dígitos a la derecha se considera decimal
    if ("," in s) or ("." in s):
        # normalizar: dejar solo el último como decimal
        last = max(s.rfind(","), s.rfind("."))
        intp = re.sub(r"[.,]", "", s[:last])
        frac = re.sub(r"[.,]", "", s[last+1:])
        s = f"{intp}.{frac}" if frac != "" else intp
    s = re.sub(r"[^0-9.\-]", "", s)
    try:
        return float(s)
    except:
        return pd.NA

pib_limpio["pib_const_2015_miles_mm"] = pib_limpio["pib_const_2015_miles_mm"].map(coerce_final_num)
pib_limpio["pib_const_2015_miles_mm"] = pd.to_numeric(pib_limpio["pib_const_2015_miles_mm"], errors="coerce").astype("Float64")

print("dtype final:", pib_limpio["pib_const_2015_miles_mm"].dtype)  # debe ser Float64

# ---- Guardados: machine & excel ----------
os.makedirs(os.path.dirname(out_base), exist_ok=True)

# 1) CSV “machine-friendly” (coma, punto decimal)
machine_csv = f"{out_base}.csv"
pib_limpio.to_csv(machine_csv, index=False, encoding="utf-8-sig", float_format="%.2f")
print("💾 Guardado (machine):", machine_csv)

# 2) CSV “excel-friendly ES” (punto y coma; decimal coma)
# excel_csv = f"{out_base}_excel.csv"
# pib_limpio.to_csv(excel_csv, index=False, encoding="utf-8-sig", sep=";", decimal=",", float_format="%.2f")
# print("💾 Guardado (excel):  ", excel_csv)



dtype final: Float64
💾 Guardado (machine): C:\Estudios\Talento_Tech\Proyecto_Talento_Tech\proyecto_movilidad_electrica\datos\limpios\pib_departamental_const2015_limpio.csv


 ### 3. CARGA  Y LIMPIEZA DE POBLACIÓN 

In [5]:
# =========================================
# CNPV 2018 — Población por departamento (fila "TOTAL")
# Salida final: departamento (estandarizado), poblacion_2018, COD_DEPTO (string)
# =========================================
import os, csv, re
import pandas as pd
import unicodedata

ruta_personas = os.path.join(ruta_datos, "Personas.csv")
ref_deps_path = os.path.join(ruta_salida, "departamentos_limpios.csv")  # generado previamente

# -------------------------------
# Utilidades
# -------------------------------
def quitar_tildes(s: str) -> str:
    if s is None:
        return ""
    return ''.join(c for c in unicodedata.normalize('NFD', str(s))
                   if unicodedata.category(c) != 'Mn')

def normalizar_txt_col(serie: pd.Series) -> pd.Series:
    # mayúsculas, espacios colapsados
    s = serie.astype(str).fillna("").str.strip()
    s = s.str.replace(r"\s+", " ", regex=True).str.upper()
    return s

def clave_union_dep(serie: pd.Series) -> pd.Series:
    """
    Clave robusta para unir departamentos:
    - mayúsculas + trim + colapsar espacios
    - quitar diacríticos
    - dejar solo letras y espacios
    """
    s = normalizar_txt_col(serie)
    s = s.apply(lambda x: ''.join(ch for ch in unicodedata.normalize('NFKD', x)
                                  if not unicodedata.combining(ch)))
    s = s.str.replace(r"[^A-Z\s]", "", regex=True)
    s = s.str.replace(r"\s+", " ", regex=True).str.strip()
    return s

# patrón para "A": puede traer prefijo numérico + separador + nombre
REG_DEP = r"^(?:\d{1,3}[-_\s]+)?([A-ZÁÉÍÓÚÜÑ ,\.]+)$"
EXCLUIR_A = {
    "TOTAL NACIONAL",
    "CABECERA",
    "RURAL DISPERSO",
    "CABECERA Y CENTROS POBLADOS Y RURAL DISPERSO",
    "CENTROS POBLADOS Y RURAL DISPERSO",
}

# -------------------------------
# 1) Detectar separador y leer sin encabezado
# -------------------------------
with open(ruta_personas, "r", encoding="utf-8", errors="replace") as fh:
    muestra = fh.read(8192)

try:
    dialect = csv.Sniffer().sniff(muestra, delimiters=[",",";","|","\t"])
    sep = dialect.delimiter
except Exception:
    sep = ";"

df = pd.read_csv(ruta_personas, sep=sep, header=None, dtype="string", engine="python")
# asegurar A..E
if df.shape[1] < 5:
    df = df.reindex(columns=range(5))
df = df.rename(columns={0:"A", 1:"B", 2:"C", 3:"D", 4:"E"})

# -------------------------------
# 2) Normalización vectorizada
# -------------------------------
df["A_norm"] = normalizar_txt_col(df["A"])
df["B_norm"] = normalizar_txt_col(df["B"])

# D viene como "5.974.788" o "44,164,417"
poblacion = (df["D"].astype(str)
               .str.replace(" ", "", regex=False)
               .str.replace(".", "", regex=False)
               .str.replace(",", "", regex=False))
df["D_num"] = pd.to_numeric(poblacion, errors="coerce").astype("Int64")

# -------------------------------
# 3) Filtro de filas: "TOTAL" por departamento
# -------------------------------
mask_total = df["B_norm"].eq("TOTAL")
mask_excluir = ~df["A_norm"].isin(EXCLUIR_A) & ~df["A_norm"].str.startswith("TOTAL NACIONAL")
mask_departamento = df["A_norm"].str.match(REG_DEP)

candidatas = df[mask_total & mask_excluir & mask_departamento].copy()

# -------------------------------
# 4) Extraer nombre de departamento (raw) y preparar clave de unión
# -------------------------------
# Tomamos el grupo 1 del patrón (nombre) de forma vectorizada, manteniendo tildes aquí;
# la clave las quita para comparar contra referencia.
candidatas["departamento_raw"] = (
    candidatas["A_norm"]
    .str.extract(REG_DEP, expand=False)
    .str.strip(" ,.")
)

# Clave de unión sin tildes/diacríticos y sin punct
candidatas["dep_join"] = clave_union_dep(candidatas["departamento_raw"])

# -------------------------------
# 4A) Cargar referencia DIVIPOLA y unir para estandarizar nombre + código
# -------------------------------
ref_deps = pd.read_csv(ref_deps_path, encoding="utf-8-sig")  # columnas: COD_DEPTO, DEPARTAMENTO
ref_deps["dep_join"] = clave_union_dep(ref_deps["DEPARTAMENTO"])

# Hacemos el merge
candidatas = candidatas.merge(
    ref_deps[["dep_join", "COD_DEPTO", "DEPARTAMENTO"]],
    on="dep_join",
    how="left"
)

# Si hay no mapeados, avisa (debug)
no_mapeados = candidatas[candidatas["COD_DEPTO"].isna()]["departamento_raw"].dropna().unique()
if len(no_mapeados) > 0:
    print("\n⚠️ Departamentos no mapeados (revisar referencia/entrada):")
    for d in no_mapeados:
        print("-", d)

# Asegurar COD_DEPTO como string con ceros
candidatas["COD_DEPTO"] = candidatas["COD_DEPTO"].astype(str).str.zfill(2)

# -------------------------------
# 5) Resultado final (estandarizado)
# -------------------------------
personas_depto = (
    candidatas.loc[:, ["DEPARTAMENTO", "D_num", "COD_DEPTO"]]
    .rename(columns={"DEPARTAMENTO": "departamento", "D_num": "poblacion_2018"})
    .dropna(subset=["departamento", "poblacion_2018"])
    .drop_duplicates(subset=["departamento"])   # por si hubiera repetidos
    .sort_values("departamento")
    .reset_index(drop=True)
)

print("=== Resultado: población por departamento (CNPV 2018, fila Total) ===")
print("Departamentos encontrados:", personas_depto["departamento"].nunique())
display(personas_depto.head(10))

# -------------------------------
# 6) Guardar archivo limpio
# -------------------------------
nombre_salida = "personas_depto.csv"
ruta_salida_archivo = os.path.join(ruta_salida, nombre_salida)

# (garantía extra) COD_DEPTO como texto 2 dígitos
personas_depto["COD_DEPTO"] = personas_depto["COD_DEPTO"].astype(str).str.zfill(2)

personas_depto.to_csv(ruta_salida_archivo, index=False, encoding="utf-8-sig")
print(f"\n💾 Archivo limpio guardado en: {ruta_salida_archivo}")




⚠️ Departamentos no mapeados (revisar referencia/entrada):
- BOYACÁ
=== Resultado: población por departamento (CNPV 2018, fila Total) ===
Departamentos encontrados: 32


,departamento,poblacion_2018,COD_DEPTO
0,AMAZONAS,66056,91.0
1,ANTIOQUIA,5974788,5.0
2,ARAUCA,239503,81.0
3,"ARCHIPIÉLAGO DE SAN ANDRÉS, PROVIDENCIA Y SANT...",48299,88.0
4,ATLÁNTICO,2342265,8.0
5,"BOGOTÁ, D.C.",7181469,11.0
6,BOLÍVAR,1909460,13.0
7,CALDAS,923472,17.0
8,CAQUETÁ,359602,18.0
9,CASANARE,379892,85.0



💾 Archivo limpio guardado en: C:\Estudios\Talento_Tech\Proyecto_Talento_Tech\proyecto_movilidad_electrica\datos\limpios\personas_depto.csv


 ### 4. CARGA  Y LIMPIEZA DE AREAS

In [9]:
# =========================================
# LIMPIEZA: ÁREAS POR DEPARTAMENTO
# Salida: codigo_depto, area_km2
# =========================================
import os, csv, re
import pandas as pd

ruta_area = os.path.join(ruta_datos, "areas_departamentos.csv")

# -- 1) Detectar separador y cargar como texto --
with open(ruta_area, "r", encoding="utf-8", errors="replace") as fh:
    muestra = fh.read(8192)
try:
    dialect = csv.Sniffer().sniff(muestra, delimiters=[",",";","|","\t"])
    sep = dialect.delimiter
except Exception:
    sep = ","

area_raw = pd.read_csv(ruta_area, sep=sep, dtype="string", engine="python")
cols_orig = area_raw.columns

# -- 2) Estandarizar nombres de columnas (lower, sin espacios) --
norm = {c: re.sub(r"\s+", "_", c.strip().lower()) for c in cols_orig}
area_raw = area_raw.rename(columns=norm)

# Candidatos para código departamento y área
cands_cod = [c for c in area_raw.columns if re.search(r"(cod|codi).*dep|^dep.*cod|^codigo_?depto|^cod_depto|^dpto(_|)cod|^codigo$", c)]
cands_area = [c for c in area_raw.columns if re.search(r"area|km2|km_?2|superficie", c)]

if not cands_cod:
    raise ValueError("No encontré columna de código de departamento.")
if not cands_area:
    raise ValueError("No encontré columna de área.")

col_cod = cands_cod[0]
col_area = cands_area[0]

df = area_raw[[col_cod, col_area]].copy()

# -- 3) Código a 2 dígitos (extrae números y zfill) --
df["codigo_depto"] = (
    df[col_cod].astype(str)
    .str.extract(r"(\d+)", expand=False)  # toma solo dígitos
    .fillna("")
    .str.zfill(2)                         # 01..99
)

# -- 4) Área numérica: limpiar sin alterar valores correctos --
s = df[col_area].astype(str).str.strip()

def limpiar_area(v):
    if pd.isna(v) or v.strip() == "":
        return None
    t = v.strip()

    # eliminar caracteres no numéricos, coma o punto
    t = re.sub(r"[^0-9\.,\-]", "", t)

    # caso típico: valor ya bien con punto decimal -> mantenerlo
    if re.match(r"^\d+\.\d+$", t):
        return t

    # caso 1: usa coma como decimal (y sin punto)
    if "," in t and "." not in t:
        t = t.replace(",", ".")
        return t

    # caso 2: tiene comas o puntos de miles (más de un separador)
    # eliminar todo menos el último punto o coma (decimal)
    if t.count(".") > 1 or t.count(",") > 1 or ("," in t and "." in t):
        # tomar el último separador como decimal
        last_sep = max(t.rfind("."), t.rfind(","))
        parte_entera = re.sub(r"[.,]", "", t[:last_sep])
        parte_decimal = re.sub(r"[.,]", "", t[last_sep + 1:])
        t = f"{parte_entera}.{parte_decimal}"
        return t

    # último recurso: quitar comas de miles
    t = t.replace(",", "")
    return t

s_limpio = s.map(limpiar_area)
df["area_km2"] = pd.to_numeric(s_limpio, errors="coerce")



# -- 5) Limpieza final: quitar vacíos y consolidar duplicados si los hay --
df = (
    df.dropna(subset=["codigo_depto", "area_km2"])
      .query("codigo_depto != ''")
      .groupby("codigo_depto", as_index=False)["area_km2"].sum()  # por si viniera desagregado
      .sort_values("codigo_depto")
      .reset_index(drop=True)
)

# -- 6) Vista rápida --
print("=== ÁREAS POR DPTO (km²) ===")
print("Departamentos:", len(df))
display(df.head(12))

# -- 7) Guardar limpio (solo columnas solicitadas) --
ruta_out = os.path.join(ruta_salida, "areas_departamentos.csv")
df.to_csv(ruta_out, index=False, encoding="utf-8-sig")
print(f"💾 Archivo limpio guardado en: {ruta_out}")






=== ÁREAS POR DPTO (km²) ===
Departamentos: 33


,codigo_depto,area_km2
0,05,62808.75
1,08,3314.46
2,11,1622.85
3,13,26720.29
4,15,23138.00
5,17,7425.22
6,18,92831.28
7,19,31242.80
8,20,22565.31
9,23,25086.28


💾 Archivo limpio guardado en: C:\Estudios\Talento_Tech\Proyecto_Talento_Tech\proyecto_movilidad_electrica\datos\limpios\areas_departamentos.csv


### 5. UNIÓN DE ARCHIVOS DE POBLACIÓN Y AREAS

In [10]:
# =========================================
# UNIÓN POBLACIÓN + ÁREA -> DENSIDAD (hab/km²)
# Salida: codigo_depto, departamento, area_km2, poblacion_2018, densidad_hab_km2
# =========================================
import os
import re
import pandas as pd
import unicodedata

def quitar_tildes(s):
    if s is None:
        return ""
    return ''.join(c for c in unicodedata.normalize('NFD', str(s))
                   if unicodedata.category(c) != 'Mn')

def norm_nombre(s):
    s = str(s).strip().upper()
    s = quitar_tildes(s)
    s = (s.replace(".", "").replace(",", "")
           .replace("  ", " ").replace("  ", " ").strip())
    return s

# --- 1) Cargar insumos limpios ---
ruta_personas_ok = os.path.join(ruta_salida, "personas_depto.csv")          # departamento, poblacion_2018
ruta_areas_ok    = os.path.join(ruta_salida, "areas_departamentos.csv")     # codigo_depto, area_km2
personas = pd.read_csv(ruta_personas_ok, dtype={"departamento":"string"})
areas    = pd.read_csv(ruta_areas_ok,    dtype={"codigo_depto":"string"})

# --- 2) Construir puente código<->nombre ---
ruta_pib_ok = os.path.join(ruta_salida, "pib_departamental_const2015_limpio.csv")
if os.path.exists(ruta_pib_ok):
    tmp = pd.read_csv(ruta_pib_ok, dtype={"codigo_depto":"string"})
    divi = tmp[["codigo_depto", "departamento"]].dropna().drop_duplicates().copy()
else:
    divi = pd.DataFrame({
        "codigo_depto": ["05","08","11","13","15","17","18","19","20","23","25","27","41","44","47",
                         "50","52","54","63","66","68","70","73","76","81","85","86","88","91","94","95","97","99"],
        "departamento": ["ANTIOQUIA","ATLANTICO","BOGOTA D C","BOLIVAR","BOYACA","CALDAS","CAQUETA","CAUCA",
                         "CESAR","CORDOBA","CUNDINAMARCA","CHOCO","HUILA","LA GUAJIRA","MAGDALENA","META","NARINO",
                         "NORTE DE SANTANDER","QUINDIO","RISARALDA","SANTANDER","SUCRE","TOLIMA","VALLE DEL CAUCA",
                         "ARAUCA","CASANARE","PUTUMAYO","SAN ANDRES PROVIDENCIA Y SANTA CATALINA","AMAZONAS",
                         "GUAINIA","GUAVIARE","VAUPES","VICHADA"]
    })

# normalizaciones para unir
divi["dep_norm"]     = divi["departamento"].map(norm_nombre)
personas["dep_norm"] = personas["departamento"].map(norm_nombre)
aliases = {
    "BOGOTA DC": "BOGOTA D C",
    "BOGOTA D C": "BOGOTA D C",
    "ARCHIPIELAGO DE SAN ANDRES PROVIDENCIA Y SANTA CATALINA": 
        "SAN ANDRES PROVIDENCIA Y SANTA CATALINA"
}
personas["dep_norm"] = personas["dep_norm"].replace(aliases)

# --- 3) Unir personas + divi, quedarnos con una sola columna 'departamento' ---
poblacion_cod = personas.merge(
    divi[["codigo_depto","dep_norm","departamento"]]
           .rename(columns={"departamento":"departamento_divi"}),
    on="dep_norm", how="left"
)

# si el puente no trae nombre, usar el de personas
poblacion_cod["departamento_personas"] = personas["departamento"].map(norm_nombre)
poblacion_cod["departamento"] = poblacion_cod["departamento_divi"].fillna(poblacion_cod["departamento_personas"])

# aseguramos tipos
poblacion_cod["codigo_depto"] = poblacion_cod["codigo_depto"].astype("string")
poblacion_cod["poblacion_2018"] = pd.to_numeric(poblacion_cod["poblacion_2018"], errors="coerce").astype("Int64")

# --- 4) Unir con áreas (por código) ---
demografia = poblacion_cod.merge(areas, on="codigo_depto", how="left")

# --- 4A) SÓLO ESTANDARIZAR NOMBRE DEPARTAMENTO SEGÚN REFERENCIA (sin tocar nada más) ---
ruta_ref_deps = os.path.join(ruta_salida, "departamentos_limpios.csv")  # columnas: COD_DEPTO, DEPARTAMENTO
ref_deps = pd.read_csv(ruta_ref_deps, encoding="utf-8-sig", dtype={"COD_DEPTO":"string"})
# asegurar 2 dígitos en el código de referencia y en el dataset actual
ref_deps["COD_DEPTO"] = ref_deps["COD_DEPTO"].astype(str).str.zfill(2)
demografia["codigo_depto"] = demografia["codigo_depto"].astype(str).str.zfill(2)

demografia = demografia.merge(
    ref_deps[["COD_DEPTO", "DEPARTAMENTO"]].rename(columns={"COD_DEPTO":"codigo_depto"}),
    on="codigo_depto", how="left"
)

# reemplazamos el nombre con el oficial (tildes correctas)
# si por alguna razón no se encuentra, conservamos el que ya teníamos
demografia["departamento"] = demografia["DEPARTAMENTO"].fillna(demografia["departamento"])
demografia = demografia.drop(columns=["DEPARTAMENTO"])

# --- 5) Calcular densidad y seleccionar columnas finales ---
demografia["area_km2"] = pd.to_numeric(demografia["area_km2"], errors="coerce")
demografia["densidad_hab_km2"] = demografia["poblacion_2018"] / demografia["area_km2"]

cols_finales = ["codigo_depto","departamento","area_km2","poblacion_2018","densidad_hab_km2"]
# si alguna no existe por nombre, ajustamos de forma segura
colmap = {c: (c if c in demografia.columns else None) for c in cols_finales}
# construir dataframe final tomando las columnas presentes
demografia_final = demografia[[c for c in cols_finales if c in demografia.columns]].copy()

# ordenar y limpiar
demografia_final = (demografia_final
                    .sort_values("codigo_depto")
                    .reset_index(drop=True))

print("=== DEMOGRAFÍA DEPARTAMENTAL ===")
print("Columnas:", list(demografia_final.columns))
print("Departamentos:", demografia_final.shape[0])
display(demografia_final.head(12))

# --- 6) Guardar ---
ruta_out = os.path.join(ruta_salida, "demografia_departamental.csv")
demografia_final.to_csv(ruta_out, index=False, encoding="utf-8-sig")
print(f"💾 Archivo limpio guardado en: {ruta_out}")



=== DEMOGRAFÍA DEPARTAMENTAL ===
Columnas: ['codigo_depto', 'departamento', 'area_km2', 'poblacion_2018', 'densidad_hab_km2']
Departamentos: 32


,codigo_depto,departamento,area_km2,poblacion_2018,densidad_hab_km2
0,05,ANTIOQUIA,62808.75,5974788,95.13
1,08,ATLÁNTICO,3314.46,2342265,706.68
2,13,BOLÍVAR,26720.29,1909460,71.46
3,17,CALDAS,7425.22,923472,124.37
4,18,CAQUETÁ,92831.28,359602,3.87
5,19,CAUCA,31242.80,1243503,39.8
6,20,CESAR,22565.31,1098577,48.68
7,23,CÓRDOBA,25086.28,1555596,62.01
8,25,CUNDINAMARCA,22370.37,2792877,124.85
9,27,CHOCÓ,48353.09,457412,9.46


💾 Archivo limpio guardado en: C:\Estudios\Talento_Tech\Proyecto_Talento_Tech\proyecto_movilidad_electrica\datos\limpios\demografia_departamental.csv


### 6. UBICACIÓN ESTACIONES EPM

In [18]:
# =========================================
# 6) LIMPIEZA SIMPLE: ESTACIONES EPM (ANTIOQUIA)
# Solo dejamos: tipo_estacion, estacion, ciudad, latitud, longitud
# =========================================

import os
import pandas as pd
import numpy as np

# ---------- Config ----------
archivo_entrada = "Estaciones_de_Gas_Natural_Vehicular_y_Carga_Eléctrica_–_EPM_20250926.csv"  # AJUSTA si tu archivo se llama diferente
ruta_in = os.path.join(ruta_datos, archivo_entrada)
ruta_out = os.path.join(ruta_salida, "estaciones_epm_antioquia.csv")

# ---------- Lectura con encodings comunes ----------
ultimo_error = None
for enc in ["utf-8-sig", "utf-8", "latin-1", "cp1252"]:
    try:
        estaciones = pd.read_csv(ruta_in, encoding=enc)
        break
    except FileNotFoundError:
        raise FileNotFoundError(f"No se encontró el archivo: {ruta_in}")
    except Exception as e:
        ultimo_error = e
else:
    raise RuntimeError(f"No pude leer el archivo {ruta_in}. Último error: {ultimo_error}")

# ---------- Arreglo básico de mojibake en NOMBRES de columnas ----------
def fix_mojibake_colname(s: str) -> str:
    rep = {
        "Ã¡": "á", "Ã©": "é", "Ãí": "í", "Ã­": "í", "Ã³": "ó", "Ãº": "ú",
        "Ã±": "ñ", "Ã‘": "Ñ", "Ã“": "Ó", "Ã‰": "É", "Ã": "Í"  # fallback común
    }
    for k,v in rep.items():
        s = s.replace(k, v)
    return s

estaciones.columns = [fix_mojibake_colname(c) for c in estaciones.columns]

# Normalizamos a minúsculas sin espacios extra
cols_map = {c: c.strip().lower() for c in estaciones.columns}
estaciones.rename(columns=cols_map, inplace=True)

# ---------- Detección de columnas de interés ----------
# Aliases que cubren tus nombres exactos y variantes habituales
ALIAS_TIPO     = {"tipo de estacion", "tipo de estación", "tipo_estacion", "tipo_estación", "tipo"}
ALIAS_NOMBRE   = {"estacion", "estación"}
ALIAS_CIUDAD   = {"ciudad", "municipio"}
ALIAS_LAT      = {"latitud", "latitude", "lat"}
ALIAS_LON      = {"longitud", "longitude", "lon"}

def pick(df_cols, alias_set):
    for a in alias_set:
        if a in df_cols:
            return a
    return None

df_cols = set(estaciones.columns)

c_tipo = pick(df_cols, ALIAS_TIPO)
c_nom  = pick(df_cols, ALIAS_NOMBRE)
c_ciu  = pick(df_cols, ALIAS_CIUDAD)
c_lat  = pick(df_cols, ALIAS_LAT)
c_lon  = pick(df_cols, ALIAS_LON)

faltantes = [n for n,(c) in {
    "tipo_estacion": c_tipo, "estacion": c_nom,
    "ciudad": c_ciu, "latitud": c_lat, "longitud": c_lon
}.items() if c is None]

if faltantes:
    raise KeyError(
        "No encontré estas columnas requeridas en el CSV: "
        + ", ".join(faltantes)
        + f"\nColumnas disponibles: {sorted(estaciones.columns)}"
    )

# ---------- Subset + renombre estándar ----------
df = estaciones[[c_tipo, c_nom, c_ciu, c_lat, c_lon]].copy()
df.columns = ["tipo_estacion", "estacion", "ciudad", "latitud", "longitud"]

# ---------- Limpieza mínima ----------
# Strip textos
for c in ["tipo_estacion", "estacion", "ciudad"]:
    df[c] = df[c].astype(str).str.strip()

# Lat/Lon a float (corrige coma decimal)
def to_float(s):
    if pd.isna(s): return np.nan
    s = str(s).strip().replace(",", ".")
    try:
        return float(s)
    except:
        return np.nan

df["latitud"] = df["latitud"].map(to_float)
df["longitud"] = df["longitud"].map(to_float)

# Quitar filas sin coordenadas válidas
df = df.dropna(subset=["latitud", "longitud"]).reset_index(drop=True)

# ---------- Guardado ----------
os.makedirs(ruta_salida, exist_ok=True)
df.to_csv(ruta_out, index=False, encoding="utf-8-sig")


print("✅ Estaciones EPM (Antioquia) — LIMPIEZA LISTA")
print(f"Filas finales: {len(df)}")
print(f"Guardado en: {ruta_out}")

display(df.head(10))



✅ Estaciones EPM (Antioquia) — LIMPIEZA LISTA
Filas finales: 55
Guardado en: C:\Estudios\Talento_Tech\Proyecto_Talento_Tech\proyecto_movilidad_electrica\datos\limpios\estaciones_epm_antioquia.csv


,tipo_estacion,estacion,ciudad,latitud,longitud
0,Estación de carga eléctrica EPM,Aeropuerto,Rionegro,6.18,-75.44
1,Estación de carga eléctrica EPM,Éxito Poblado,Medellín,6.21,-75.57
2,Estación de carga eléctrica EPM,Éxito Poblado,Medellín,6.21,-75.57
3,Estación de carga eléctrica EPM,Exposiciones,Medellín,6.24,-75.58
4,Estación de carga eléctrica EPM,EDS Texaco Vegas,Envigado,6.18,-75.59
5,Estación de carga eléctrica EPM,1er Parque Laureles,Medellín,6.25,-75.59
6,Estación de carga eléctrica EPM,1er Parque Laureles,Medellín,6.25,-75.59
7,Estación de carga eléctrica EPM,CC Los Molinos,Medellín,6.23,-75.60
8,Estación de carga eléctrica EPM,CC Los Molinos,Medellín,6.23,-75.60
9,Estación de carga eléctrica EPM,CC Mayorca,Sabaneta,6.16,-75.60


### Dataset maestro de vehiculos y pib

In [19]:
# -*- coding: utf-8 -*-
"""
Dataset 2: Vehículos registrados (EV / HEV) + PIB
Columnas finales:
anio, codigo_departamento, departamento, ev_registrados, hev_registrados, pib_const_2015
"""

import pandas as pd
import numpy as np
from pathlib import Path

# =========================
# 1) Rutas
# =========================
BASE = Path(r"C:\Estudios\Talento_Tech\Proyecto_Talento_Tech\proyecto_movilidad_electrica\datos\limpios")

VEH_PATH = BASE / "vehiculos_ev_hev_limpio.csv"
PIB_PATH = BASE / "pib_departamental_const2015_limpio.csv"
OUT_PATH = BASE / "vehiculos_pib.csv"

# =========================
# 2) Utilidades
# =========================
def read_csv_robust(path: Path) -> pd.DataFrame:
    if not Path(path).exists():
        raise FileNotFoundError(f"No se encontró el archivo: {path}")
    for enc in ("utf-8", "latin1", "cp1252"):
        try:
            return pd.read_csv(path, encoding=enc)
        except Exception:
            continue
    return pd.read_csv(path)

def normalize_cols(df: pd.DataFrame) -> pd.DataFrame:
    out = df.copy()
    out.columns = (
        out.columns.str.strip().str.lower()
        .str.replace(r"\s+", "_", regex=True)
        .str.replace(r"[^a-z0-9_]", "", regex=True)
    )
    return out

def strip_accents_series(s: pd.Series) -> pd.Series:
    return (
        s.astype(str)
         .str.normalize("NFKD")
         .str.encode("ascii", errors="ignore")
         .str.decode("utf-8")
    )

def norm_dep(series: pd.Series) -> pd.Series:
    s = strip_accents_series(series.fillna("").astype(str).str.lower().str.strip())
    repl = {
        "bogota dc": "bogota",
        "bogota, d.c.": "bogota",
        "bogota d.c.": "bogota",
        "archipielago de san andres, providencia y santa catalina": "san andres y providencia",
        "san andres": "san andres y providencia",
    }
    return s.replace(repl)

def coerce_year(series: pd.Series) -> pd.Series:
    y = series.astype(str).str.extract(r"(\d{4})", expand=False)
    return pd.to_numeric(y, errors="coerce").astype("Int64")

def fix_code_str(s: pd.Series) -> pd.Series:
    code = s.astype(str).str.extract(r"(\d+)", expand=False)
    return code.where(code.isna(), code.str.zfill(2))

def first_present(df: pd.DataFrame, names) -> str | None:
    for n in names:
        if n in df.columns:
            return n
    return None

# =========================
# 3) Vehículos: EV y HEV por anio-departamento
# =========================
veh = normalize_cols(read_csv_robust(VEH_PATH))

dep_col  = first_present(veh, ["departamento","departamento_nombre","depto","dpto"])
anio_col = first_present(veh, ["anio","ano","año","anio_registro","anioinscripcion","anio_modelo","year","periodo"])
fecha_col = first_present(veh, ["fecha_registro","fecha","fec_registro"])
cod_col  = first_present(veh, ["codigo_departamento","codigo_depto","cod_departamento","cod_depto","cod_dane","codigo_dane"])
tipo_col = first_present(veh, ["tipo_vehiculo","tipo","categoria"])

if not (dep_col and tipo_col):
    raise ValueError(f"Vehículos: faltan columnas clave (departamento/tipo). Columnas: {veh.columns.tolist()}")

veh["_departamento_"] = norm_dep(veh[dep_col])
veh["_cod_depto_"] = fix_code_str(veh[cod_col]) if cod_col else pd.NA

# Año: usar columna de año si existe; si no, extraer de la fecha
if anio_col:
    veh["_anio_"] = coerce_year(veh[anio_col])
elif fecha_col:
    veh["_anio_"] = coerce_year(veh[fecha_col])
else:
    raise ValueError("Vehículos: no se encontró columna de año ni fecha para derivarlo.")

# Conteo: si hay columna de cantidad, úsala; si no, cuenta filas
cant_col = first_present(veh, ["cantidad","n","count","conteo","valor"])
veh["_cnt_"] = pd.to_numeric(veh[cant_col], errors="coerce").fillna(0) if cant_col else 1

# Normalizar tipo (EV / HEV)
tipo_norm = veh[tipo_col].astype(str).str.upper().str.strip()
veh["_tipo_"] = np.select(
    [
        tipo_norm.str.contains(r"\bEV\b") & ~tipo_norm.str.contains("HEV|PHEV"),
        tipo_norm.str.contains(r"\bHEV\b"),
    ],
    ["EV","HEV"],
    default="OTRO"
)

veh_grp = (
    veh.groupby(["_anio_","_cod_depto_","_departamento_","_tipo_"], dropna=False)["_cnt_"]
       .sum()
       .reset_index()
)

veh_piv = veh_grp.pivot_table(
    index=["_anio_","_cod_depto_","_departamento_"],
    columns="_tipo_",
    values="_cnt_",
    aggfunc="sum",
    fill_value=0
).reset_index()
veh_piv.columns.name = None

veh_piv["ev_registrados"]  = veh_piv.get("EV", 0)
veh_piv["hev_registrados"] = veh_piv.get("HEV", 0)
veh_min = veh_piv[["_anio_","_cod_depto_","_departamento_","ev_registrados","hev_registrados"]].copy()

# =========================
# 4) PIB
# =========================
pib = normalize_cols(read_csv_robust(PIB_PATH))

dep_pib = first_present(pib, ["departamento","departamento_nombre","depto","dpto"])
anio_pib = first_present(pib, ["anio","ano","año","anio_registro","year","periodo"])
cod_pib  = first_present(pib, ["codigo_departamento","codigo_depto","cod_departamento","cod_depto","cod_dane","codigo_dane"])
pib_col  = next((c for c in pib.columns if "pib" in c), None)

if not (dep_pib and (anio_pib or fecha_col) and pib_col):
    raise ValueError(f"PIB: faltan columnas (departamento/anio/pib). Columnas: {pib.columns.tolist()}")

pib["_departamento_"] = norm_dep(pib[dep_pib])
pib["_anio_"] = coerce_year(pib[anio_pib]) if anio_pib else pd.NA
pib["_cod_depto_"] = fix_code_str(pib[cod_pib]) if cod_pib else pd.NA
pib["pib_const_2015"] = pd.to_numeric(pib[pib_col], errors="coerce")

pib_min = pib[["_anio_","_cod_depto_","_departamento_","pib_const_2015"]].copy()

# =========================
# 5) MERGE y columnas finales
# =========================
df = veh_min.merge(pib_min, on=["_anio_","_cod_depto_","_departamento_"], how="left")

# Si quedó PIB nulo, intentar completar por (anio + nombre)
mask = df["pib_const_2015"].isna()
if mask.any():
    alt = pib_min.drop(columns=["_cod_depto_"]).drop_duplicates(["_anio_","_departamento_"])
    df.loc[mask, "pib_const_2015"] = df[mask].merge(
        alt, on=["_anio_","_departamento_"], how="left"
    )["pib_const_2015_y"].values

df["anio"] = df["_anio_"]
df["codigo_departamento"] = df["_cod_depto_"].replace({"<NA>": np.nan})
df["departamento"] = df["_departamento_"].str.upper()

dataset = df[[
    "anio","codigo_departamento","departamento",
    "ev_registrados","hev_registrados","pib_const_2015"
]].sort_values(["departamento","anio"]).reset_index(drop=True)

# =========================
# 6) Guardado + chequeos
# =========================
OUT_PATH.parent.mkdir(parents=True, exist_ok=True)
dataset.to_csv(OUT_PATH, index=False, encoding="utf-8")

print(">>> Guardado:", OUT_PATH)
print("Shape:", dataset.shape)
print("Columnas:", list(dataset.columns))
print("NaNs por columna:\n", dataset.isna().sum())
if {"departamento","anio"}.issubset(dataset.columns):
    print("Duplicados (departamento, anio):", dataset.duplicated(subset=["departamento","anio"]).sum())



>>> Guardado: C:\Estudios\Talento_Tech\Proyecto_Talento_Tech\proyecto_movilidad_electrica\datos\limpios\vehiculos_pib.csv
Shape: (212, 6)
Columnas: ['anio', 'codigo_departamento', 'departamento', 'ev_registrados', 'hev_registrados', 'pib_const_2015']
NaNs por columna:
 anio                   0
codigo_departamento    0
departamento           0
ev_registrados         0
hev_registrados        0
pib_const_2015         0
dtype: int64
Duplicados (departamento, anio): 0
